In [0]:
from pyspark.sql import SparkSession, functions as f

#Reading Hospital A departments data
df_hosa= spark.read.parquet("dbfs:/mnt/bronze/hosa/providers")

#Reading Hospital B departments data
df_hosb= spark.read.parquet("dbfs:/mnt/bronze/hosb/providers")

#union two departments dataframes
df_merged= df_hosa.unionByName(df_hosb)
display(df_merged)



df_merged.createOrReplaceTempView("providers")

In [0]:
display(df_hosa)

In [0]:
%sql


CREATE TABLE if not exists silver.providers (
  ProviderID string,
  FirstName string,
  LastName string,
  Specialization string,
  DeptID string,
  NPI long,
  datasource string,
  is_quarantined BOOLEAN
)
USING DELTA

In [0]:
%sql
truncate table silver.departments;

In [0]:
%sql
INSERT INTO silver.providers
SELECT distinct
    ProviderID,
    FirstName,  
    LastName,
    Specialization,
    DeptID,
    cast(NPI as INT) as NPI,
    datasource,
    CASE 
        WHEN ProviderID IS NULL OR DeptID IS NULL THEN true
        ELSE false
    END as is_quarantined
FROM providers

In [0]:
%sql

select * from silver.providers 